In [1]:
# Load data from the processed data file
import pickle
import os
import pandas as pd

PROCESSED_DATA_DIR = "processed"
LOADED_DATA = {}

with open(os.path.join(PROCESSED_DATA_DIR, 'geolife_processed_data.pkl'), 'rb') as f:
    LOADED_DATA = pickle.load(f)


for participant, dataframes in LOADED_DATA.items():
    for label, df in dataframes.items():
        print(f"Participant: {participant}, Label: {label}")
        print(df.info())
        print(df.head())
        break
    break

Participant: 1, Label: 0_bus
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   72 non-null     float64       
 1   longitude  72 non-null     float64       
 2   date_time  72 non-null     datetime64[ns]
 3   timestamp  72 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 2.4 KB
None
    latitude   longitude           date_time     timestamp
0  39.975133  116.329467 2007-06-27 09:49:22  1.182938e+09
1  39.975250  116.329067 2007-06-27 09:50:07  1.182938e+09
2  39.975267  116.329283 2007-06-27 09:50:11  1.182938e+09
3  39.975250  116.329417 2007-06-27 09:50:33  1.182938e+09
4  39.975283  116.330033 2007-06-27 09:51:58  1.182938e+09


In [2]:
from dataset import GeoLifeMobilityDataset